In [1]:
from pathlib import Path
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
import yfinance as yf
import numpy as np

##### Portfolio

In [2]:
# Load portfolio data
df_holding = pd.read_excel('portfolio.xls') 
tickers = df_holding['Ticker'].tolist()
shares = df_holding.set_index('Ticker')['Shares']

# Create monthly date range
dates = pd.date_range(start='2017-10-31', end='2023-12-31', freq='M')

# Download and process price data for all tickers
price_data = {}
for ticker in tickers:
    try:
        # Download data with buffer period
        data = yf.download(ticker, start='2017-10-01', end='2023-12-31', auto_adjust=True, progress=False)
        data.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
        
        # Handle missing data
        if data.empty:
            print(f"No data found for {ticker}, filling with zeros")
            price_data[ticker] = pd.Series(0, index=dates)
            continue

        # Resample to monthly closing prices with forward fill
        monthly_prices = data['Close'].resample('M').last().ffill()
        price_data[ticker] = monthly_prices.reindex(dates, method='ffill').fillna(0)

    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        price_data[ticker] = pd.Series(0, index=dates)

# Create price DataFrame
prices = pd.DataFrame(price_data)

# Calculate monthly returns
monthly_returns = prices.pct_change()

# Adjust shares based on monthly return conditions
adjusted_shares = shares.copy()

for date in monthly_returns.index[1:]:  # Skip the first month (no return data)
    for ticker in tickers:
        if monthly_returns.at[date, ticker] > 0.20:
            adjusted_shares[ticker] *= 0.8  # Reduce shares by 20%
        elif monthly_returns.at[date, ticker] < -0.20:
            adjusted_shares[ticker] *= 1.1  # Increase shares by 10%

    # Calculate market values with adjusted shares
    market_values = prices.loc[date] * adjusted_shares

    # Update total portfolio value
    if 'total_value' not in locals():
        total_value = pd.Series(0, index=monthly_returns.index)  # Initialize total value series
    total_value.at[date] = market_values.sum()

# Calculate monthly percentage change
portfolio_return = total_value.pct_change() * 100

# Create final performance DataFrame
portfolio_df = pd.DataFrame({
    'Date': total_value.index,
    'Portfolio_Value': total_value.values,
    'Portfolio_Return': portfolio_return.values
}).set_index('Date')

portfolio_df.dropna(inplace=True)
portfolio_df = portfolio_df[~portfolio_df.isin([np.inf, -np.inf]).any(axis=1)]

In [3]:
portfolio_df

,Portfolio_Value,Portfolio_Return
Date,,
2017-12-31,2.570644e+06,1.850332
2018-01-31,2.688656e+06,4.590763
2018-02-28,2.655385e+06,-1.237461
2018-03-31,2.611731e+06,-1.643975
2018-04-30,2.597190e+06,-0.556761
...,...,...
2023-08-31,4.192476e+06,-3.137686
2023-09-30,3.976659e+06,-5.147704
2023-10-31,4.016135e+06,0.992680


##### Macro Factors

In [4]:
# Define the directory where the CSV files are located
directory = Path('C:/Users/Kathe/Documents/Python/Quantitative Finance/Multi-Factor Risk Model for Equity Portfolio/')

# Create an empty list to store dataframes
dataframes = []

# Loop through each CSV file in the directory that starts with 'macro'
for file_path in directory.glob('macro*.csv'):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Extract file name for dynamic naming
    file_name = file_path.stem.lower()

    # Ensure index is in datetime format
    df['observation_date'] = pd.to_datetime(df['observation_date'])
    df.set_index('observation_date', inplace=True)

    # Resample, fill gaps, and sort properly
    df = df.resample('M').mean(numeric_only=True).interpolate(method='linear', limit_direction='both').sort_index()

    # Identify the last column dynamically
    value_column_name = df.columns[-1]

    # Calculate change metrics
    if 'cpi' in file_name:
        df['Inflation'] = df[value_column_name].pct_change()
    elif 'treasury' in file_name:
        df['Yield_Change'] = df[value_column_name].diff()

    # Drop the 'observation_date' column if it exists
    df.drop(columns=['observation_date'], errors='ignore', inplace=True)

    # Append the processed dataframe to the list
    dataframes.append(df)

# Combine all dataframes into one
macro_df = pd.concat(dataframes)

# Filter data between '2018-01-01' and '2023-12-31'
macro_df = macro_df.sort_index().loc['2017-12-01':'2023-12-31']

# Forward fill missing values and drop remaining NaNs
macro_df = macro_df.groupby(macro_df.index).ffill()
macro_df.dropna(inplace=True)

# Rename columns based on content
colname_dict = {col: 'CPI' if 'cpi' in col.lower() else 'GDP_Growth' if 'nbea' in col.lower() else col for col in macro_df.columns}
macro_df.rename(columns=colname_dict, inplace=True)

In [5]:
macro_df

,CPI,Inflation,GDP_Growth,T10Y2Y,Yield_Change
observation_date,,,,,
2017-12-31,247.805,0.002107,3.2,0.558500,-0.098167
2018-01-31,248.859,0.004253,3.3,0.551905,-0.006595
2018-02-28,249.529,0.002692,3.3,0.683684,0.131779
2018-03-31,249.577,0.000192,3.3,0.566667,-0.117018
2018-04-30,250.227,0.002604,3.3,0.484762,-0.081905
...,...,...,...,...,...
2023-08-31,306.138,0.005000,3.2,-0.733913,0.195087
2023-09-30,307.374,0.004037,3.2,-0.643000,0.090913
2023-10-31,307.653,0.000908,3.2,-0.269524,0.373476


##### Style Factors

In [6]:
# Define the start and end dates
start = dt.datetime(2017, 12, 1)
end = dt.datetime(2023, 12, 31)

# Fetch Fama-French 3-factor data (Monthly data)
fama_french_df = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start, end)[0]

# Ensure index is in datetime format, and shows the end of a month
fama_french_df.index = fama_french_df.index.to_timestamp() + pd.offsets.MonthEnd(0)

In [7]:
fama_french_df

,Mkt-RF,SMB,HML,RF
Date,,,,
2017-12-31,1.06,-1.32,0.06,0.09
2018-01-31,5.57,-3.12,-1.28,0.12
2018-02-28,-3.65,0.26,-1.04,0.11
2018-03-31,-2.35,4.06,-0.20,0.11
2018-04-30,0.29,1.13,0.54,0.14
...,...,...,...,...
2023-08-31,-2.39,-3.20,-1.08,0.45
2023-09-30,-5.24,-2.49,1.45,0.43
2023-10-31,-3.18,-3.88,0.19,0.47


##### Sector Factors

In [8]:
# Initialize a dictionary to store data by sector
sector_dict = {}

# Function to calculate RSI
def calculate_rsi(prices, period=14):
    """Calculate Relative Strength Index (RSI)"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Loop over each ticker to download data and calculate metrics
for ticker in tickers:
    try:
        # Download historical data (starting earlier to calculate RSI from 2018)
        data = yf.download(ticker, start='2017-01-01', end='2023-12-31', auto_adjust=True, progress=False)
        if data.empty:
            print(f"No data available for {ticker}")
            continue
        data.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
        
        # Get sector information from Yahoo Finance
        try:
            sector_info = yf.Ticker(ticker).info.get('sector', 'N/A')  # 'N/A' if sector not found
        except Exception as e:
            sector_info = 'N/A'
            print(f"Error retrieving sector for {ticker}: {e}")
        
        # Calculate monthly close prices and RSI
        monthly_close = data['Close'].resample('M').last()
        monthly_rsi = calculate_rsi(data['Close']).resample('M').last()  # Monthly RSI
        
        # Calculate 30-day volatility (annualized) from daily returns
        data['Daily_Return'] = data['Close'].pct_change()
        data['30D_Vol'] = data['Daily_Return'].rolling(30).std() * np.sqrt(252)
        monthly_volatility = data['30D_Vol'].resample('M').last()
        
        # Combine into a DataFrame
        df_ticker = pd.DataFrame({
            'Date': monthly_close.index,
            'RSI': monthly_rsi.values,  # Replace 12M_Momentum with RSI
            '30dVol': monthly_volatility.values
        }).set_index('Date')

        # Append the ticker data to the sector data
        if sector_info not in sector_dict:
            sector_dict[sector_info] = []
        sector_dict[sector_info].append(df_ticker)
        
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        continue

# Aggregate results by sector (average across tickers)
sector_combined = {}
for sector, dfs in sector_dict.items():
    if not dfs:  # Skip sectors with no data
        continue
    df_sector = pd.concat(dfs)
    # Group by date and average metrics
    df_sector_avg = df_sector.groupby('Date').mean()
    sector_combined[sector] = df_sector_avg

# Combine all sectors into a multi-index DataFrame
sector_df = pd.concat(sector_combined, axis=1)

# Filter dates between 2018-01-01 and 2023-12-31
sector_df = sector_df.loc['2017-12-01':'2023-12-31']

# Format column names for multi-index
sector_df.columns.names = ['Sector', 'Metric']

In [9]:
sector_df

Sector     Technology           Healthcare           Financial Services  \
Metric            RSI    30dVol        RSI    30dVol                RSI   
Date                                                                      
2017-12-31  55.070970  0.213540  55.932184  0.120019          57.600704   
2018-01-31  60.108755  0.163216  61.282159  0.335006          73.044449   
2018-02-28  69.827147  0.300953  57.129557  0.426411          56.774088   
2018-03-31  39.311392  0.327076  24.771445  0.325424          32.187404   
2018-04-30  44.392113  0.355113  61.367712  0.334296          34.810601   
...               ...       ...        ...       ...                ...   
2023-08-31  66.039058  0.243931  31.846052  0.178193          29.649595   
2023-09-30  28.520241  0.243762  45.993987  0.168945          41.612203   
2023-10-31  41.337598  0.236113  34.086721  0.234712          42.401002   
2023-11-30  68.967094  0.226277  66.371086  0.222269          82.571901   
2023-12-31  45.687628  0.183405  60.710598  0.225063          76.411882   

Sector               Consumer Cyclical           Communication Services  \
Metric        30dVol               RSI    30dVol                    RSI   
Date                                                                      
2017-12-31  0.184491         37.446877  0.152210              65.010012   
2018-01-31  0.149954         28.772887  0.170391              70.135493   
2018-02-28  0.307221         66.836712  0.212250              56.693008   
2018-03-31  0.274850         46.485934  0.187117              33.340754   
2018-04-30  0.295879         36.309519  0.229219              39.088427   
...              ...               ...       ...                    ...   
2023-08-31  0.188701         37.072181  0.184699              59.775248   
2023-09-30  0.172726         27.955868  0.156065              35.321016   
2023-10-31  0.248588         51.496364  0.163715              62.583729   
2023-11-30  0.279987         39.917434  0.329768              87.189871   
2023-12-31  0.239202         46.828766  0.166736              57.722472   

Sector               Consumer Defensive           Industrials            \
Metric        30dVol                RSI    30dVol         RSI    30dVol   
Date                                                                      
2017-12-31  0.222821          66.320487  0.333484   67.195053  0.223841   
2018-01-31  0.288398          75.838897  0.139430   32.204463  0.207890   
2018-02-28  0.390810          26.098323  0.384667   52.855988  0.370465   
2018-03-31  0.323044          50.833879  0.367126   43.038586  0.271902   
2018-04-30  0.375494          60.479654  0.215200   56.356442  0.249906   
...              ...                ...       ...         ...       ...   
2023-08-31  0.253262          54.184014  0.115348   27.558349  0.327412   
2023-09-30  0.256943          30.365052  0.113901   34.340659  0.141787   
2023-10-31  0.430211          67.382587  0.137257   47.047666  0.184628   
2023-11-30  0.384315          35.844586  0.272379   76.497509  0.184049   
2023-12-31  0.194590          72.204006  0.272977   76.602811  0.168878   

Sector     Basic Materials           Real Estate            Utilities  \
Metric                 RSI    30dVol         RSI    30dVol        RSI   
Date                                                                    
2017-12-31       87.748792  0.167264   65.689871  0.131932  39.265407   
2018-01-31       50.851262  0.214085   75.176242  0.182568  59.416994   
2018-02-28       50.333771  0.340322   53.147525  0.237116  39.933622   
2018-03-31       45.375422  0.302574   48.553558  0.224624  61.878602   
2018-04-30       57.792198  0.287471   41.518637  0.256753  47.390603   
...                    ...       ...         ...       ...        ...   
2023-08-31       43.818339  0.272075   28.301917  0.191819  48.707994   
2023-09-30       51.717756  0.279673   34.484982  0.194518  34.574496   
2023-10-31       48.025612  0.261323   46.32

##### Final Dataset

In [10]:
final_df = pd.concat([portfolio_df, macro_df, fama_french_df, sector_df], axis=1)
final_df.dropna(inplace=True)
final_df.to_csv("final_data.csv")